<a href="https://colab.research.google.com/github/bae3559/2021summerintern/blob/main/vibe_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VIBE: Video Inference for Human Body Pose and Shape Estimation

Demo of the original PyTorch based implementation provided here: https://github.com/mkocabas/VIBE

## Note
Before running this notebook make sure that your runtime type is 'Python 3 with GPU acceleration'. Go to Edit > Notebook settings > Hardware Accelerator > Select "GPU".

## More Info
- Paper: https://arxiv.org/abs/1912.05656
- Repo: https://github.com/mkocabas/VIBE

In [1]:
# Clone the repo
!git clone https://github.com/mkocabas/VIBE.git

Cloning into 'VIBE'...
remote: Enumerating objects: 397, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 397 (delta 8), reused 4 (delta 0), pack-reused 375
Receiving objects: 100% (397/397), 15.11 MiB | 3.98 MiB/s, done.
Resolving deltas: 100% (179/179), done.


In [12]:
%cd VIBE/

[Errno 2] No such file or directory: 'VIBE/'
/content/VIBE


In [3]:
# Install the other requirements
!pip install torch==1.4.0 numpy==1.17.5
!pip install git+https://github.com/giacaglia/pytube.git --upgrade
!pip install -r requirements.txt

  Cloning https://github.com/giacaglia/pytube.git to /tmp/pip-req-build-6tzns8ab
  Running command git clone -q https://github.com/giacaglia/pytube.git /tmp/pip-req-build-6tzns8ab
  Created wheel for pytube: filename=pytube-9.5.3-cp37-none-any.whl size=32290 sha256=8d2f3ba00d1a038ecb501dfab6f1131ac161555252b2b884d47eebdc4a58d6f2
  Stored in directory: /tmp/pip-ephem-wheel-cache-4uqrx_s6/wheels/49/5a/fe/342957c87dc4c1e1a244fbeffcdbf8a0f2c8db0277823f3bfd
Successfully built pytube
  Found existing installation: pytube 9.5.3
    Uninstalling pytube-9.5.3:
      Successfully uninstalled pytube-9.5.3
  Cloning https://github.com/mattloper/chumpy.git to /tmp/pip-req-build-ws44i2va
  Running command git clone -q https://github.com/mattloper/chumpy.git /tmp/pip-req-build-ws44i2va
  Cloning https://github.com/mkocabas/yolov3-pytorch.git to /tmp/pip-req-build-srdegpz9
  Running command git clone -q https://github.com/mkocabas/yolov3-pytorch.git /tmp/pip-req-build-srdegpz9
  Cloning https://github

In [4]:
# Download pretrained weights and SMPL data
!source /content/VIBE/scripts/prepare_data.sh

Downloading...
From: https://drive.google.com/uc?id=1untXhYOLQtpNEy4GTY_0fL_H-k6cTf_r
To: /content/VIBE/data/vibe_data.zip
561MB [00:06, 91.3MB/s]
Archive:  vibe_data.zip
   creating: vibe_data/
  inflating: vibe_data/smpl_mean_params.npz  
  inflating: vibe_data/vibe_model_w_3dpw.pth.tar  
  inflating: vibe_data/gmm_08.pkl    
  inflating: vibe_data/J_regressor_h36m.npy  
  inflating: vibe_data/vibe_model_wo_3dpw.pth.tar  
  inflating: vibe_data/SMPL_NEUTRAL.pkl  
  inflating: vibe_data/J_regressor_extra.npy  
  inflating: vibe_data/spin_model_checkpoint.pth.tar  
  inflating: vibe_data/sample_video.mp4  
  inflating: vibe_data/yolov3.weights  


### Run the demo code.

Check https://github.com/mkocabas/VIBE/blob/master/doc/demo.md for more details about demo.

**Note:** Final rendering is slow compared to inference. We use pyrender with GPU accelaration and it takes 2-3 FPS per image. Please let us know if you know any faster alternative. 

In [7]:
# Run the demo
!python demo.py --vid_file dance.mp4 --output_folder output/ --sideview

# You may use --sideview flag to enable from a different viewpoint, note that this doubles rendering time.
# !python demo.py --vid_file sample_video.mp4 --output_folder output/ --sideview

# You may also run VIBE on a YouTube video by providing a link
# python demo.py --vid_file https://www.youtube.com/watch?v=c4DAnQ6DtF8 --output_folder output/ --display

Running "ffmpeg -i dance.mp4 -f image2 -v error /tmp/dance_mp4/%06d.png"
Images saved to "/tmp/dance_mp4"
Input video number of frames 6286
Running Multi-Person-Tracker
100% 524/524 [04:34<00:00,  2.10it/s]
Finished. Detection + Tracking FPS 22.86
=> loaded pretrained model from 'data/vibe_data/spin_model_checkpoint.pth.tar'
Performance of pretrained model on 3DPW: 56.56075477600098
Loaded pretrained weights from "data/vibe_data/vibe_model_wo_3dpw.pth.tar"
Running VIBE on each tracklet...
100% 5/5 [01:41<00:00, 21.44s/it]
VIBE FPS: 61.79
Total time spent: 399.98 seconds (including model loading time).
Total FPS (including model loading time): 15.72.
Saving output results to "output/dance/vibe_output.pkl".
Rendering output video, writing frames to /tmp/dance_mp4_output
100% 6286/6286 [25:59<00:00,  4.18it/s]
Saving result video to output/dance/dance_vibe_result.mp4
Running "ffmpeg -y -threads 16 -i /tmp/dance_mp4_output/%06d.png -profile:v baseline -level 3.0 -c:v libx264 -pix_fmt yuv42

In [15]:
# Play the generated video
from IPython.display import HTML
from base64 import b64encode

def video(path):
  mp4 = open(path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML('<video width=500 controls loop> <source src="%s" type="video/mp4"></video>' % data_url)

video('output/dance/dance_vibe_result.mp4')

In [ ]:
# Inspect the output file content
import joblib
output = joblib.load('output/sample_video/vibe_output.pkl')
print('Track ids:', output.keys(), end='\n\n')

print('VIBE output file content:', end='\n\n')
for k,v in output[1].items():
  if k != 'joints2d': 
    print(k, v.shape)

###SPIN

In [16]:
%cd /content/
!unzip --q SPIN-master.zip


/content
Archive:  SPIN-master.zip
5c796852ca7ca7373e104e8489aa5864323fbf84
   creating: SPIN-master/
  inflating: SPIN-master/.gitignore  
  inflating: SPIN-master/LICENSE     
  inflating: SPIN-master/README.md   
  inflating: SPIN-master/config.py   
  inflating: SPIN-master/constants.py  
   creating: SPIN-master/data/
  inflating: SPIN-master/data/README.md  
   creating: SPIN-master/datasets/
  inflating: SPIN-master/datasets/__init__.py  
  inflating: SPIN-master/datasets/base_dataset.py  
  inflating: SPIN-master/datasets/mixed_dataset.py  
   creating: SPIN-master/datasets/preprocess/
  inflating: SPIN-master/datasets/preprocess/README.md  
  inflating: SPIN-master/datasets/preprocess/__init__.py  
  inflating: SPIN-master/datasets/preprocess/coco.py  
  inflating: SPIN-master/datasets/preprocess/h36m.py  
  inflating: SPIN-master/datasets/preprocess/h36m_train.py  
  inflating: SPIN-master/datasets/preprocess/hr_lspet.py  
  inflating: SPIN-master/datasets/preprocess/lsp_data

In [26]:
%cd /content/SPIN-master

import os
os.environ["PYOPENGL_PLATFORM"] = "egl"

!python3 demo.py --checkpoint=data/model_checkpoint.pt --img=examples/im1010.jpg --openpose=examples/im1010_openpose.json



/content/SPIN-master
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/OpenGL/platform/osmesa.py", line 25, in GL
    mode=ctypes.RTLD_GLOBAL 
  File "/usr/local/lib/python3.7/dist-packages/OpenGL/platform/ctypesloader.py", line 45, in loadLibrary
    return dllType( name, mode )
  File "/usr/lib/python3.7/ctypes/__init__.py", line 364, in __init__
    self._handle = _dlopen(self._name, mode)
OSError: ('OSMesa: cannot open shared object file: No such file or directory', 'OSMesa', None)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "demo.py", line 36, in <module>
    from utils.renderer import Renderer
  File "/content/SPIN-master/utils/renderer.py", line 6, in <module>
    import pyrender
  File "/usr/local/lib/python3.7/dist-packages/pyrender/__init__.py", line 3, in <module>
    from .light import Light, PointLight, DirectionalLight, SpotLight
  File "/usr/local/lib/python3.7/dist-packag

In [22]:
from smplx.body_models import SMPLOutput

